## **演示0703：多变量线性回归**

### **提出问题**
在之前的实验中，披萨价格仅与直径有关，按照这一假设，其预测的结果并不令人满意(R方=0.662)。本章再引入一个新的影响因素：披萨辅料级别(此处已经把辅料级别调整成数值，以便能够进行数值计算)。训练数据如下：

![](../images/070301.png)

另外提供测试数据如下：

![](../images/070302.png)

如何使用线性回归训练数据，并且判断是否有助于提升预测效果呢？

### **案例1：基于LinearRegression的实现**
>  
* 与单变量线性回归类似，但要注意训练数据此时是(是训练数据条数，是自变量个数)，在本例中，是5x2的矩阵：<code>xTrain = np.array([[6,2],[8,1],[10,0],[14,2],[18,0]])</code>
* 针对测试数据的预测结果，其R方约为0.77，已经强于单变量线性回归的预测结果

In [1]:
''' 使用LinearRegression进行多元线性回归 '''

import numpy as np
from sklearn.linear_model import LinearRegression

xTrain = np.array([[6, 2], [8, 1], [10, 0], [14, 2], [18, 0]])  # 无需手动添加Intercept Item项         
yTrain = np.array([7, 9, 13, 17.5, 18])

xTest= np.array([[8, 2], [9, 0], [11, 2], [16, 2], [12, 0]])
yTest = np.array([11, 8.5, 15, 18, 11])

model = LinearRegression()
model.fit(xTrain, yTrain)
hpyTest = model.predict(xTest)

print("假设函数参数：", model.intercept_, model.coef_)
print("测试数据预测结果与实际结果差异：", hpyTest - yTest)
print("测试数据R方：", model.score(xTest, yTest))

假设函数参数： 1.1875 [1.01041667 0.39583333]
测试数据预测结果与实际结果差异： [-0.9375      1.78125    -1.90625     0.14583333  2.3125    ]
测试数据R方： 0.7701677731318468


### **案例2：基于成本函数和梯度下降的实现**
对于一个自变量$x_1$的情形，$y$与$x$的关系用一条直线就可以拟合(假设有一定线性相关性)。对于有两个自变量$x_1, x_2$的情形，$y$与$x$的关系就需要用一个平面来拟合。如果有更多的自变量，虽然无法在三维空间中展现，但仍然可以用数学的方式来描述它们之间的关系。
* 判别函数：$ h_\theta(x)=\theta_0 x_0+ \theta_1 x_1+ \theta_2 x_2+ \cdots + \theta_n x_n $
 * $x_0$称为Inercept Term，一般设置为1即可
 * $x_1,x_2,\cdots,x_n$表示影响$y$的各个因素。假设共有$n$个影响因素(即$n$个维度)
* 判别函数的矩阵表述形式：  
对于每一个样本，都有：  
$ h_\theta(x^{(1)})=\theta_0 x_0^{(1)}+ \theta_1 x_1^{(1)}+ \theta_2 x_2^{(1)}+ \cdots + \theta_n x_n^{(1)} $  
$ h_\theta(x^{(2)})=\theta_0 x_0^{(2)}+ \theta_1 x_1^{(2)}+ \theta_2 x_2^{(2)}+ \cdots + \theta_n x_n^{(2)} $  
$\cdots \cdots$  
$ h_\theta(x^{(m)})=\theta_0 x_0^{(m)}+ \theta_1 x_1^{(m)}+ \theta_2 x_2^{(m)}+ \cdots + \theta_n x_n^{(m)} $  
其中：$x_j^{(i)}$ 表示第$i$个样本数据的第$j$个自变量(第$j$个维度)。注意，各组数据的第0个自变量均为Intercept Term，直接设置为1  
以矩阵运算的方式表示上面的各组公式，可得：    
$ h_\theta=X * \theta $  
其中：  
$ X=\left(\begin{matrix}
1 & x_1^{(1)} & x_2^{(1)} & \cdots & x_n^{(1)} \\
1 & x_1^{(2)} & x_2^{(2)} & \cdots & x_n^{(2)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_1^{(m)} & x_2^{(m)} & \cdots & x_n^{(m)}
\end{matrix}\right) $，$ \theta=\left(\begin{matrix}
\theta_0 \\
\theta_1 \\
\vdots \\
\theta_n
\end{matrix}\right) $
* 成本函数的矩阵运算形式：  
$ J(\theta)= \dfrac{1}{2m} \sum_{i=1}^m(h_\theta (x^{(i)}) - y^{(i)})^2 =\dfrac{1}{2m} (X * \theta - y)^T (X * \theta - y) $
* 梯度变化(偏导数):  
$ \begin{aligned}
\dfrac{\partial J(\theta)}{\partial \theta_0} & =
\dfrac{\partial [\dfrac{1}{2m} \sum_{i=1}^m (\theta_0 + \theta_1 x_1^{(i)} + \cdots + \theta_n x_n^{(i)})^2 ]}{\partial \theta_0} \\ \\ & =
\dfrac{1}{m} \sum_{i=1}^m (\theta_0 + \theta_1 x_1^{(i)} + \cdots + \theta_n x_n^{(i)}) * x_0^{(i)}
\end{aligned} $  
$ \begin{aligned}
\dfrac{\partial J(\theta)}{\partial \theta_1} & =
\dfrac{\partial [\dfrac{1}{2m} \sum_{i=1}^m (\theta_0 + \theta_1 x_1^{(i)} + \cdots + \theta_n x_n^{(i)})^2 ]}{\partial \theta_1} \\ \\ & =
\dfrac{1}{m} \sum_{i=1}^m (\theta_0 + \theta_1 x_1^{(i)} + \cdots + \theta_n x_n^{(i)}) * x_1^{(i)}
\end{aligned} $  
$ \vdots $  
$ \begin{aligned}
\dfrac{\partial J(\theta)}{\partial \theta_n} & =
\dfrac{\partial [\dfrac{1}{2m} \sum_{i=1}^m (\theta_0 + \theta_1 x_1^{(i)} + \cdots + \theta_n x_n^{(i)})^2 ]}{\partial \theta_n} \\ \\ & =
\dfrac{1}{m} \sum_{i=1}^m (\theta_0 + \theta_1 x_1^{(i)} + \cdots + \theta_n x_n^{(i)}) * x_n^{(i)}
\end{aligned} $  
上式中，$ x_0^{(i)} = 1 $  
将上述各组式子用矩阵形式表达如下：  
$ \dfrac{\partial J(\theta)}{\partial \theta}=X^T * (X * \theta - y) $

In [2]:
''' 批量梯度下降法实现多元线性回归 '''

import numpy as np
import matplotlib.pyplot as plt
import bgd_resolver

def costFn(theta, X, y):                           # 成本函数
    temp = X.dot(theta) - y
    return (temp.T.dot(temp)) / (2 * len(X))

def gradientFn(theta, X, y):                       # 根据成本函数，分别对x0,x1...xn求导数(梯度)
    return (X.T).dot(X.dot(theta) - y) / len(X)


xTrainData = np.array([[6, 2], [8, 1], [10, 0], [14, 2], [18, 0]])
yTrain = np.array([7, 9, 13, 17.5, 18])
xTrain = np.c_[xTrainData, np.ones(len(xTrainData))]

np.random.seed(0)
init_theta = np.random.randn(xTrain.shape[1])
theta = bgd_resolver.batch_gradient_descent(costFn, gradientFn, init_theta, xTrain, yTrain) 
print("theta值", theta)

xTestData = np.array([[8, 2], [9, 0], [11, 2], [16, 2], [12, 0]])
yTest = np.array([11, 8.5, 15, 18, 11])
xTest = np.c_[xTestData, np.ones(len(xTestData))]
print("测试数据预测值与真实值的差异：", xTest.dot(theta) - yTest)

rsquare = bgd_resolver.batch_gradient_descent_rsquare(theta, xTest, yTest)
print("测试数据R方：", rsquare)

循环 40961 次后收敛
theta值 [1.01128555 0.39906853 1.17356105]
测试数据预测值与真实值的差异： [-0.93801751  1.77513099 -1.90416086  0.15226688  2.30898763]
测试数据R方： 0.7709259763685181
